In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

# Cross Validation

It is a statistical technique used in machine learning and statistics to assess how well a predictive model will perform on unseen data. The basic idea is to split the available dataset into multiple subsets or folds.

[Cross Validation Explained!](https://youtu.be/a86WxNgMv7E?si=3RVFsQgAViQsCL-z)  
[Machine Learning Fundamentals: Cross Validation](https://youtu.be/fSytzGwwBVw?si=TSZXN41ZNLQPXm5U)

## Types of Cross Validation

1. K-Fold Cross Validation
2. Stratified K-Fold Cross Validation
3. Leave One Out Cross Validation
4. Repeated K-Fold Cross Validation
5. Shuffle Split Cross Validation
6. Time Series Cross Validation

[Complete Guide to Cross Validation](https://youtu.be/-8s9KuNo5SA?si=rk1Ltp9_5GpqYiX_)

In [2]:
from sklearn.datasets import load_digits
digits = load_digits()

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(digits.data,
                                                    digits.target,
                                                    test_size=0.3,
                                                    random_state=2002)

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [5]:
lr = LogisticRegression(solver='liblinear',multi_class='ovr')
lr.fit(X_train, y_train)
print(lr.score(X_test, y_test))

svm = SVC(gamma='auto')
svm.fit(X_train, y_train)
print(svm.score(X_test, y_test))

rf = RandomForestClassifier(n_estimators=40)
rf.fit(X_train, y_train)
print(rf.score(X_test, y_test))

0.9648148148148148
0.4
0.9740740740740741


### K-Fold Cross Validation

The model is trained on k-1 folds and validated on the remaining fold. This process is repeated k times, with each fold serving as the validation set exactly once.

In [6]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=7,shuffle=True)
kf

KFold(n_splits=7, random_state=None, shuffle=True)

In [7]:
print(' '*25+'Train Set'+' '*34+'Test Set'+' '*5)
for train_index, test_index in kf.split(np.random.randint(-50,150,25)):
    print(train_index, test_index)

                         Train Set                                  Test Set     
[ 0  1  2  3  4  5  6  8  9 10 13 14 15 16 17 18 19 20 21 22 24] [ 7 11 12 23]
[ 0  1  2  3  5  7  8  9 10 11 12 13 14 15 16 19 20 21 22 23 24] [ 4  6 17 18]
[ 1  2  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 21 22 23 24] [ 0  3 19 20]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 17 18 19 20 21 22 23 24] [13 14 15 16]
[ 0  1  2  3  4  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 22 23] [ 5 21 24]
[ 0  1  3  4  5  6  7  9 10 11 12 13 14 15 16 17 18 19 20 21 23 24] [ 2  8 22]
[ 0  2  3  4  5  6  7  8 11 12 13 14 15 16 17 18 19 20 21 22 23 24] [ 1  9 10]


In [8]:
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

In [9]:
scores_logistic = []
scores_svm = []
scores_rf = []

for train_index, test_index in kf.split(digits.data,digits.target):
    X_train, X_test, y_train, y_test = digits.data[train_index], digits.data[test_index], \
                                       digits.target[train_index], digits.target[test_index]
    scores_logistic.append(get_score(LogisticRegression(solver='liblinear',multi_class='ovr'), X_train, X_test, y_train, y_test))  
    scores_svm.append(get_score(SVC(gamma='auto'), X_train, X_test, y_train, y_test))
    scores_rf.append(get_score(RandomForestClassifier(n_estimators=40), X_train, X_test, y_train, y_test))

In [10]:
scores_logistic

[0.9766536964980544,
 0.9688715953307393,
 0.9688715953307393,
 0.953307392996109,
 0.9416342412451362,
 0.953125,
 0.9609375]

### Stratified K-Fold Cross Validation

This technique is similar to k-fold cross-validation, but it ensures that each fold has approximately the same proportion of target classes as the original dataset. It's particularly useful for imbalanced datasets where one class is much more prevalent than the others.

In [13]:
from sklearn.model_selection import cross_val_score
cross_val_score(RandomForestClassifier(n_estimators=37),X_train,y_train,cv=5)

array([0.90938511, 0.86688312, 0.93831169, 0.96428571, 0.92207792])

### Leave One Out Cross Validation

This process is repeated n times, where n is the total number of observations in the dataset. It is computationally expensive, especially for large datasets, but it provides a less biased estimate of the model's performance.

In [16]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()

In [17]:
for train_index, test_index in loo.split(np.random.randint(-50,150,10)):
    print(train_index, test_index)

[1 2 3 4 5 6 7 8 9] [0]
[0 2 3 4 5 6 7 8 9] [1]
[0 1 3 4 5 6 7 8 9] [2]
[0 1 2 4 5 6 7 8 9] [3]
[0 1 2 3 5 6 7 8 9] [4]
[0 1 2 3 4 6 7 8 9] [5]
[0 1 2 3 4 5 7 8 9] [6]
[0 1 2 3 4 5 6 8 9] [7]
[0 1 2 3 4 5 6 7 9] [8]
[0 1 2 3 4 5 6 7 8] [9]


### Shuffle Split Cross Validation

This process is repeated multiple times. It is useful when the dataset is too large to be easily divided into folds or when you want a specific number of iterations rather than a fixed number of folds.

In [18]:
from sklearn.model_selection import ShuffleSplit
ss = ShuffleSplit()

In [19]:
for train_index, test_index in ss.split(np.random.randint(-50,150,10)):
    print(train_index, test_index)

[2 9 5 7 1 3 0 4 6] [8]
[7 0 5 1 6 9 3 2 4] [8]
[0 1 8 3 2 5 7 9 6] [4]
[7 3 9 6 5 1 0 8 2] [4]
[4 6 9 7 3 8 2 1 5] [0]
[9 4 1 3 5 6 8 7 0] [2]
[8 3 2 0 9 1 7 4 5] [6]
[8 7 6 0 3 2 1 5 9] [4]
[0 5 1 9 6 7 4 3 8] [2]
[4 1 7 3 2 6 9 5 0] [8]


### Repeated Shuffle Split Cross Validation

This method involves repeating k-fold cross-validation multiple times with different random splits of the data. It helps to reduce the variability in the estimated performance of the model.

In [21]:
from sklearn.model_selection import RepeatedStratifiedKFold
rsk = RepeatedStratifiedKFold()

In [22]:
for train_index, test_index in ss.split(np.random.randint(-50,150,10)):
    print(train_index, test_index)

[6 1 4 0 2 9 5 3 7] [8]
[7 9 1 4 0 8 2 5 6] [3]
[5 1 2 6 3 9 7 0 4] [8]
[3 7 8 0 4 9 2 1 5] [6]
[4 0 2 1 7 8 5 6 9] [3]
[7 1 8 5 4 9 0 6 3] [2]
[6 9 1 3 4 8 5 7 0] [2]
[8 2 6 0 7 5 9 4 3] [1]
[7 0 2 1 4 6 5 3 8] [9]
[6 4 2 3 5 8 1 7 9] [0]


### Time Series Cross Validation

Time series data requires a different approach to cross-validation because the order of observations matters. Techniques like forward chaining or sliding window validation are commonly used for time series data, where the model is trained on past data and validated on future data.

[Time Series Cross Validation](https://www.youtube.com/watch?v=g9iO2AwTXyI)  
https://www.youtube.com/live/355u2bDqB7c?si=7XXIl24LAPfD71ox&t=4303

## Hyperparameter Tuning

It is the process of finding the best settings for a machine learning model, like choosing the right learning rate or number of trees. It's crucial for optimizing the model's performance on unseen data.

[GridSearchCV | Hyperparameter Tuning | Machine Learning with Scikit-Learn Python](https://youtu.be/TvB_3jVIHhg?si=SWqHZ2z_zYonOgNb)

In [23]:
link = 'https://raw.githubusercontent.com/daaanishhh002/MachineLearning/main/Datasets/heart.csv'
df = pd.read_csv(link)

df.sample(5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
284,61,1,0,140,207,0,0,138,1,1.9,2,1,3,0
111,57,1,2,150,126,1,1,173,0,0.2,2,1,3,1
99,53,1,2,130,246,1,0,173,0,0.0,2,3,2,1
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
245,48,1,0,124,274,0,0,166,0,0.5,1,0,3,0


In [24]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,:-1],
                                                 df.iloc[:,-1],
                                                 test_size=0.3,
                                                 random_state=2002)

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [26]:
rf = RandomForestClassifier()
lr = LogisticRegression()
knn = KNeighborsClassifier()

In [27]:
knn.fit(X_train,y_train)
knn.score(X_test,y_test)

0.6593406593406593

In [28]:
lr.fit(X_train,y_train)
lr.score(X_test,y_test)

0.7802197802197802

In [29]:
rf.fit(X_train,y_train)
rf.score(X_test,y_test)

0.7472527472527473

In [30]:
rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [31]:
rf = RandomForestClassifier(max_samples=0.75,random_state=2002)

rf.fit(X_train,y_train)
rf.score(X_test,y_test)

0.7582417582417582

In [32]:
from sklearn.model_selection import cross_val_score
np.mean(cross_val_score(RandomForestClassifier(),df.iloc[:,:-1],
                                                 df.iloc[:,-1],cv=10,scoring='accuracy'))

0.8148387096774193

### Grid Search CV

In [33]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [34]:
n_estimators = [20,60,100,120]
max_features = [0.2,0.6,1.0]
max_depth = [2,8,None]
max_samples = [0.5,0.75,1.0]

In [35]:
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
              'max_samples':max_samples
}

In [36]:
rf = RandomForestClassifier()

rf_grid = GridSearchCV(estimator = rf, 
                       param_grid = param_grid, 
                       cv = 5, 
                       verbose=2, 
                       n_jobs = -1)

In [37]:
rf_grid.fit(X_train,y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [2, 8, None],
                         'max_features': [0.2, 0.6, 1.0],
                         'max_samples': [0.5, 0.75, 1.0],
                         'n_estimators': [20, 60, 100, 120]},
             verbose=2)

In [38]:
rf_grid.best_score_

0.8681063122923588

In [39]:
rf_grid.best_params_

{'max_depth': 8, 'max_features': 0.2, 'max_samples': 0.5, 'n_estimators': 100}

### Random Search CV

In [40]:
n_estimators = [20,60,100,120]
max_features = [0.2,0.6,1.0]
max_depth = [2,8,None]
max_samples = [0.5,0.75,1.0]
bootstrap = [True,False]
min_samples_split = [2, 5]
min_samples_leaf = [1, 2]

In [41]:
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
              'max_samples':max_samples,
              'bootstrap':bootstrap,
              'min_samples_split':min_samples_split,
              'min_samples_leaf':min_samples_leaf
}

In [42]:
from sklearn.model_selection import RandomizedSearchCV

rf_grid = RandomizedSearchCV(estimator = rf, 
                       param_distributions = param_grid,
                       n_iter=10,
                       cv = 5, 
                       verbose=2, 
                       n_jobs = -1)

In [43]:
rf_grid.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [2, 8, None],
                                        'max_features': [0.2, 0.6, 1.0],
                                        'max_samples': [0.5, 0.75, 1.0],
                                        'min_samples_leaf': [1, 2],
                                        'min_samples_split': [2, 5],
                                        'n_estimators': [20, 60, 100, 120]},
                   verbose=2)

In [44]:
rf_grid.best_score_

0.8582502768549279

In [45]:
rf_grid.best_params_

{'n_estimators': 60,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_samples': 1.0,
 'max_features': 0.6,
 'max_depth': None,
 'bootstrap': True}